# End to End 프로젝트 실습

## 1. 단계별로 각각의 작업들 확인
터미널을 연뒤에 현재 디렉토리로 이동 하고 다음 명령어들을 실행해봅니다!

In [ ]:
#혹시 기존에 6006포트를 사용하고 있는 프로세스가 있다면 다음 명령어로 종료를 해주겠습니다.
#kill -9 $(lsof -t -i:6006)

### 1-1. 환경변수 설정
- .bash_profile이나 .bash_rc 같은 곳에 영구적으로 설정 할 수 있으나, 저희는 새로운 터미널을 열때마다 직접 설정을 해주겠습니다.
- 현재 디렉토리가 daily_mission_airflow 인 상태에서 다음 명령어를 입력합니다

In [ ]:
# export WORKING_DIRECTORY=$(pwd)
# export AIRFLOW_HOME=~/airflow

### 1-2. 기존의 훈련데이터를 데이터 베이스(sqlite)에 적재
- 이 부분은 최초 1회만 실행됩니다
- 실제 서비스에서는 MySQL과 같은 RDB나 NoSQL 계열의 진짜 DB를 사용하지만, 추가적인 환경설정에 힘을 빼지 않기 위해 저희는 [sqlite](https://en.wikipedia.org/wiki/SQLite)를 사용합니다.

In [ ]:
# python file_to_db.py --path ${WORKING_DIRECTORY}/event_data.csv

### 1-3. 데이터를 DB에서 추출해서 csv형태로 저장 (학습을 위한 데이터)
이때, 데이터가 지속적으로 유입된다는 가상의 시나리오를 위해 한번 실행시 10000행씩 추출을 합니다.

In [ ]:
#python ${WORKING_DIRECTORY}/db_to_file.py

### 1-4. 훈련
- 위에서 준비된 새로운 데이터 파일을 기반으로 훈련을 진행합니다
- 기존의 train_data.csv가 아니고, 지속적으로 유입되는 10000개의 데이터라고 가정을 하기 위함입니다.

In [ ]:

# python ${WORKING_DIRECTORY}/dkt/train.py --model_dir ${WORKING_DIRECTORY}/models --asset_dir ${WORKING_DIRECTORY}/asset --data_dir ${WORKING_DIRECTORY} --file_name data.csv



wandb에서 현재 훈련과정이 정상적으로 기록됐는지 확인해주세요!
단, 소량의 데이터로만 훈련을 진행해서 무의미한 acc, auc가 나올 수 있습니다!

### 1-5. 서버실행
- 우선 웹서비스 자체가 잘 동작하는지 다음 명령어로 확인해보겠습니다.
- 일단 처음엔는 airflow GUI를 확인해야 하기 때문에, 외부에서 접근 가능하지 않은 포트로 일단 돌려 놓기만 하겠습니다.

In [ ]:
# python ${WORKING_DIRECTORY}/server/server.py --port 6007

### 1-6. 서버 재시작
새로운 모델 파일로 다시 서버를 로딩해야하는 경우, 정석적으로 서버를 운영할때는 시스템 데몬 형태로 운영하여서, 해당 데몬을 재부팅 하는 식으로 진행하지만, 저희는 flask에서 제공하는 debug기능을 활용하여 약간의 트릭을 사용하겠습니다.

In [ ]:
#touch -m ${WORKING_DIRECTORY}/server/server.py

## 2. Airflow 셋팅

In [ ]:
#이미 세팅을 하신 경우에는 생략하셔도 됩니다!
!pip uninstall sqlalchemy -y
!pip install 'sqlalchemy < 1.4.0' apache-airflow attrdict mlflow
!airflow db init
!mkdir ~/airflow/dags
#GUI를 위한 유저생성

!airflow users create \
    --username admin \
    --firstname Peter \
    --lastname Parker \
    --password 1234 \
    --role Admin \
    --email spiderman@superhero.org

In [ ]:
#터미널을 하나 켜서 다음 명령어 입력
# export TZ=Asia/Seoul
# export AIRFLOW_HOME=~/airflow
# export WORKING_DIRECTORY=$(pwd)
# airflow scheduler

In [ ]:
#터미널을 하나 더 켜서 다음 명령어 입력 (Airflow GUI 실행)
# export TZ=Asia/Seoul
# export AIRFLOW_HOME=~/airflow
# export WORKING_DIRECTORY=$(pwd)
# airflow webserver -p 6006

# 이제 다음 주소로 접근 가능합니다 (텐서보드 접속 포트를 확인해주세요!)
# http://<SERVER_IP>:6009

In [ ]:
%%writefile ~/airflow/dags/dkt_pipeline.py
from datetime import timedelta
from airflow import DAG
from airflow.operators.bash import BashOperator

from airflow.utils.dates import days_ago



default_args = {
    'owner': 'Peter Parker',
    'email': ['example@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=2),
}





dag = DAG(
    'dkt_pipeline',
    default_args=default_args,
    description='DKT workflow management',
    start_date=days_ago(0),
    schedule_interval='*/1 * * * *',
    is_paused_upon_creation=False,
    catchup = False,
    max_active_runs=1
)

dataload = BashOperator(
    task_id='dataload',
    bash_command='python ${WORKING_DIRECTORY}/db_to_file.py',
    dag=dag,
)

train = BashOperator(
    task_id='train',
    bash_command='python ${WORKING_DIRECTORY}/dkt/train.py \
                    --model_dir ${WORKING_DIRECTORY}/models \
                    --asset_dir ${WORKING_DIRECTORY}/asset \
                    --data_dir ${WORKING_DIRECTORY} --file_name data.csv',
    dag=dag,
)

reload = BashOperator(
    task_id='reload',
    bash_command="touch -m ${WORKING_DIRECTORY}/server/server.py",
    dag=dag,
)


dataload >> train >> reload


In [ ]:
# db를 reset 해주고 터미널로가서 scheduler를 다시 실행합니다.
!airflow db reset -y

## 3. 입력데이터가 반영된 재훈련 확인
airflow가 정상적으로 작업을 하는 것을 확인했으면, 이제 airflow webserver를 종료하고, 웹사이트의 포트를 변경한 뒤에 다시 여러분들의 문제풀이 데이터를 입력해주세요!

In [ ]:
# python ${WORKING_DIRECTORY}/server/server.py --port 6006